## Mapas coropléticos

Um [mapa coroplético](https://en.wikipedia.org/wiki/Choropleth_map) usa regiões sombreadas ou texturizadas para visualizar os valores dos dados. Mapas de símbolos dimensionados são comumente  mais precisos para leitura, pois as pessoas tendem a ser melhores em estimar diferença proporcional entre áreas de círculos à tonalidades de cor. No entanto, mapas coropléticos são populares na prática e especialmente úteis quando muitos símbolos ficam visualmente sobrecarregados.

Por exemplo, enquanto nos EUA têm apenas 50 estados, existem milhares de condados dentro desses estados. Vamos fazer um mapa coroplético da taxa de desemprego por condado durante o ano de recessão de 2008. Em alguns casos, arquivos de entrada no formato GeoJSON ou TopoJSON podem incluir dados estatísticos que podemos visualizar diretamente. Neste caso, porém, temos 2 arquivos: um arquivo TopoJSON que contém os limites dos condados (`usa`) e um arquivo de texto separado com as estátisticas de desemprego.

In [22]:
unemp = data.unemployment.url
unemp

'https://vega.github.io/vega-datasets/data/unemployment.tsv'

Para integrar nossas fontes de dados, nós precisaremos denovo usar a transformação procurar `(lookup)`, aumentando os dados `geoshape` baseados em TopoJSON com as taxas de desemprego. Em seguida, podemos criar um mapa que inclua uma codificação para cor `(color)` para o campo taxa`(rate)` obtido na busca.

In [23]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True), 
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Examine as taxas de desemprego por condado. Valores mais altos em Michigan podem estar relacionados a industria automotiva. condados nas [grandes planícies](https://en.wikipedia.org/wiki/Great_Plains) e nos estados montanhosos exibem tanto taxas baixas **quanto** altas. Essa variação é significativa ou pode ser um [artefato de tamanhos amostrais menores](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)? Para explorar mais, tente mudar o domínio da escala superior (por exemplo, para `0.2`) para ajustar o mapeamento de cores.*

Uma preocupação central para o mapa coroplético é a escolha de cores. No exemplo acima, nós usamos o esquema padrão do altair amareloverdeazul`(yellowgreenblue)` de mapas de calor. Abaixo, comparamos outros esquemas de cores, incluindo: **nota para revisores: o trecho abaixo era texto corrido mas optei por fazer assim, se acharem melhor voltar podem fazer**
 - Um esquema sequencial de tom único (teals), que varia apenas em luminância.
 - Um esquema sequencial multicolorido (viridis), que varia em luminância e matiz.
 - Um esquema divergente (blueorange), que utiliza um ponto médio em branco. 

In [24]:
# utility function to generate a map specification for a provided color scheme
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

alt.HConcatChart(...)

_Quais esquemas de cores você achou mais eficazes? por que isso acontece? Modifique os mapas acima para usar outros esquemas disponíveis, descritos em [Vega esquema de cores documentação](https://vega.github.io/vega/docs/schemes/)._

## Projeções Cartográficas

Agora que temos alguma experiência criando mapas, vamos analisar mais de perto as projeções cartográficas. Como explicado pela [Wikipedia](https://en.wikipedia.org/wiki/Map_projection),

> _Todas as projeções de mapas necessariamente distorcem a superfície de alguma forma. Dependendo do objetivo do mapa, algumas distorções são aceitáveis e outras não; portanto, existem diferentes projeções de mapa para preservar algumas propriedades do corpo esférico em detrimento de outras propriedades._

Algumas das propriedades que podemos considerar incluem:

- _Área_: A projeção distorce o tamanho das regiões?
- _Rumo_ : Uma linha reta corresponde a uma direção constante de viagem?
- _Distância_: Linhas de igual comprimento correspondem a distâncias iguais no globo?
- _Forma_: A projeção preserva as relações espaciais (ângulos) entre os pontos? 

Selecionar uma projeção apropriada depende, portanto, do propósito do mapa. Por exemplo, se estamos avaliando o uso da terra e a extensão da terra for importante, podemos escolher uma projeção que preserve as áreas. Se quisermos visualizar ondas de choque de um terremoto, podemos focar o mapa no epicentro do terremoto e preservar as distâncias a partir desse ponto. Ou, se quisermos ajudar na navegação, a preservação do rumo e da forma pode ser mais importante.

Também podemos caracterizar as projeções em termos da _superfície de projeção_. Por exemplo, as projeções cilíndricas projetam pontos de superfície da esfera em torno de um cilindro; o cilindro "desenrolado" então fornece nosso mapa. Como descrevemos abaixo, podemos alternativamente projetar na superfície de um cone (projeções cônicas) ou diretamente em um plano liso (projeções azimutais).

Vamos primeiro desenvolver nossa intuição interagindo com uma variedade de projeções! **[abra o capítulo online de projeções cartográficas do Vega-Lite](https://observablehq.com/@vega/vega-lite-cartographic-projections).** Use os controles dessa página para selecionar uma projeção e explorar parâmetros de projeção, como `escala` (zoom) e tradução x/y (panorama). os controles da rotação ([yaw,pitch,roll](https://en.wikipedia.org/wiki/Aircraft_principal_axes)) determinam a orientação do globo em relação à superfície sendo projetada.

### Um Tour pelos Tipos Específicos de Projeção 

As [**Projeções Cilíndricas**](https://en.wikipedia.org/wiki/Map_projection#Cylindrical) mapeiam a esfera em torno de um cilindro, para então desenrolar o cilindro. Se o eixo principal do cilindro está orientado de norte para sul, meridianos são mapeados como linhas retas. As projeções [pseudo-cilíndricas](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical) representam o meridiano central como uma linha reta, com outros meridianos "curvando-se" para longe do centro.

In [25]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='equirectangular'),
    minimap.project(type='mercator').properties(title='mercator'),
    minimap.project(type='transverseMercator').properties(title='transverseMercator'),
    minimap.project(type='naturalEarth1').properties(title='naturalEarth1')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Equirretangular](https://en.wikipedia.org/wiki/Equirectangular_projection)(`equirectangular`): Escala os valores de latitude `(lat)` e longitude `(lon)` diretamente
- [Mercador](https://en.wikipedia.org/wiki/Mercator_projection) (`Mercator`): Projeta sobre um cilindro, usando a `lon` diretamente, mas submetendo a `lat` a uma transformação não linear. As linhas retas preservam os rumos constantes ([Linhas de Rumo](https://en.wikipedia.org/wiki/Rhumb_line)), fazendo essa projeção mais adequada para navegação. No entanto, as áreas nas regiões mais ao norte ou sul podem ser bem distorcidas.
- [Mercador Transversal](https://en.wikipedia.org/wiki/Transverse_Mercator_projection) (`transverseMercator`): Uma projeção de Mercator mas com o cilindro de limite rotacionado para uma eixo transversal. Enquanto a projeção do mercador padrão tem maior precisão ao longo do equador, a projeção do mercador transversal é mais precisa ao longo do meridiano central.
- [Terra Natural](https://en.wikipedia.org/wiki/Natural_Earth_projection) (`naturalEarth1`): Uma projeção pseudo-cilíndrica projetada para mostrar a Terra inteira em uma única visão.
<br/><br/>

As [**Projeções Cônicas**](https://en.wikipedia.org/wiki/Map_projection#Conic) mapeiam a esfera em torno de um cone e, em seguida, desenrolam o cone para um plano. As projeções são configuradas por dois _paralelas padrão_, que determinam onde o cone intersecta o globo.

In [26]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='conicEqualArea'),
    minimap.project(type='conicEquidistant').properties(title='conicEquidistant'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='conicConformal'),
    minimap.project(type='albers').properties(title='albers'),
    minimap.project(type='albersUsa').properties(title='albersUsa')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Cônica de Áreas Iguais](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) (`conicConformal`): Projeção cônica que preserva a área. A forma e a distância não são preservadas, mas a precisão é razoavelmente boa dentro dos paralelos padrão.
- [Cônica Equidistantes](https://en.wikipedia.org/wiki/Albers_projection) (`conicEqualArea`): projeção cônica que preserva a distância ao longo dos meridianos e paralelas padrão.
- [Cônica Conformal](https://en.wikipedia.org/wiki/Equidistant_conic_projection) (`conicEquidistant`): projeção cônica que preserva a forma (ângulos locais), mas nem a área nem distância.
- [Albers](https://en.wikipedia.org/wiki/Albers_projection) (`albers`): Uma variante da projeção cônica que iguala a área projetada com as paralelas padrão otimizadas para a criação de mapas dos Estados Unidos.
- [Albers USA](https://en.wikipedia.org/wiki/Albers_projection) (`albersUsa`): Uma projeção híbrida para os 50 estados dos EUA. Essa projeção une três projeções Albers com parâmetros diferentes para os EUA continental, Alasca e Havai.
<br/><br/>

As [**Projeções Azimutais**](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) mapeiam a esfera diretamente para um plano.

In [27]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='azimuthalEqualArea'),
    minimap.project(type='azimuthalEquidistant').properties(title='azimuthalEquidistant'),
    minimap.project(type='orthographic').properties(title='orthographic'),
    minimap.project(type='stereographic').properties(title='stereographic'),
    minimap.project(type='gnomonic').properties(title='gnomonic')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Azimutal de Área igual](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) (`azimuthalEqualArea`): Projeta áreas com precisão em todas as partes do globo, mas não preserva a forma (ângulos locais).
- [Azimutal Equidistante](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection) (`azimuthalEquidistant`): Mantém as distâncias proporcionais entre o centro da projeção e todos os outros pontos no globo.
- [Ortográfica](https://en.wikipedia.org/wiki/Orthographic_projection_in_cartography) (`orthographic`): Projeta um hemisfério visível em um plano distante, simulando aproximadamente a visão da Terra a partir do espaço.
- [Estereográfica](https://en.wikipedia.org/wiki/Stereographic_projection) (`stereographic`): Preserva a forma, mas não mantém nem a área nem a distância.
- [gnomônica ](https://en.wikipedia.org/wiki/Gnomonic_projection) (`gnomonic`): Projeta a superfície da esfera diretamente em um plano tangente. Os [grandes círculos](https://en.wikipedia.org/wiki/Great_circle) em torno da Terra são projetados em linhas retas, mostrando o menor caminho entre pontos.
<br/><br/>

## Coda: Manipulando Dados Geográficos 

Os exemplos acima foram feitos pela coleção vega-datasets, incluindo dados geométricos (TopoJSON) e tabulares (como aeroportos e taxas de desemprego).Um desafio comum para começar a visualização geométrica é coletar os dados necessários para sua tarefa. Existem diversos provedores de dados disponíveis, incluindo serviços como: [United States Geological Survey](https://www.usgs.gov/products/data/all-data) e [U.S. Census Bureau](https://www.census.gov/data/datasets.html).

Em vários casos, você pode já possuir dados com um componente geográfico, mas precisar de medidas ou geometrias adicionais. Para ajudar você a começar, aqui está um fluxo de trabalho:

In many cases you may have existing data with a geographic component, but require additional measures or geometry. To help you get started, here is one workflow:

1. Visite [Natural Earth Data](http://www.naturalearthdata.com/downloads/) e pesquise para selecionar dados por regiões e resuloções de interesse. Baixe o(s) arquivo(s) zip correspondente(s).

2. Vá para [MapShaper](https://mapshaper.org/) e arraste seu arquivo zip para a página. Edite os dados como quiser, e em seguida, "exporte" os arquivos TopoJSON ou GeoJSON gerados.
3. Carregue os dados exportados do MapShaper para usar no Altair!

Claro, muitas outras ferramentas &ndash; de código aberto e proprietárias &ndash; existem para trabalhar com dados geográficos. Para mais sobre manipulação de dados geográficos e criação de mapas, veja a série de tutorial do Mike Bostock [Command-Line Cartography](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c).

## Resumo

Até este ponto, nós apenas arranhamos a superfície do oceano de criação de mapas. _(Você não esperava que um único capítulo transmitisse séculos de estudo, né?)_ Por exemplo, nós deixamos de lado tópicos como [_cartogramas_](https://en.wikipedia.org/wiki/Cartogram) e a [_representação de relevos_](https://en.wikipedia.org/wiki/Topography) &mdash; como no brilhante livro de Imhof [_Cartographic Relief Presentation_](https://books.google.com/books?id=cVy1Ms43fFYC). Porém, agora você deveria estar bem preparado para criar uma vasta gama de geo-visualização. Para mais, livro do MacEachren [_How Maps Work: Representation, Visualization, and Design_](https://books.google.com/books?id=xhAvN3B0CkUC) oferece uma visão geral valiosa da criação de mapas da perspectiva da visualização de dados.